In [1]:
data = [
    {'price': 850, 'rooms': 4, 'neighborhood': 'Queen Anne'},
    {'price': 650, 'rooms': 3, 'neighborhood': 'Queen Anne'},
    {'price': 700, 'rooms': 1, 'neighborhood': 'Wallingford'},
    {'price': 650, 'rooms': 3, 'neighborhood': 'Wallingford'},
    {'price': 700, 'rooms': 3, 'neighborhood': 'Fremont'},
    {'price': 600, 'rooms': 2, 'neighborhood': 'Fremont'}
]
data

[{'price': 850, 'rooms': 4, 'neighborhood': 'Queen Anne'},
 {'price': 650, 'rooms': 3, 'neighborhood': 'Queen Anne'},
 {'price': 700, 'rooms': 1, 'neighborhood': 'Wallingford'},
 {'price': 650, 'rooms': 3, 'neighborhood': 'Wallingford'},
 {'price': 700, 'rooms': 3, 'neighborhood': 'Fremont'},
 {'price': 600, 'rooms': 2, 'neighborhood': 'Fremont'}]

In [2]:
import pandas as pd    
from statsmodels.formula.api import ols as sm_ols
print('The coefs from SM:')
print(sm_ols('price ~ neighborhood - 1', d

The coefs from SM:
neighborhood[Fremont]        650.0
neighborhood[Queen Anne]     750.0
neighborhood[Wallingford]    675.0
dtype: float64


In [4]:
Maybe we do this: 
    {'Queen Anne':1, 'Fremont':2, 'Wallingford':3}

SyntaxError: invalid syntax (<ipython-input-4-910ae74ac077>, line 1)

In [3]:
# create an object ("vec") that can do the transform
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False, dtype=int) 

# apply vec with ".fit_transform", save to new data obj
data2 = vec.fit_transform(data) 
print(data2, '\n')              
print(vec.get_feature_names())  # can use .get_feature_names() to recover names

# now we can repeat the regression here
from sklearn.linear_model import LinearRegression
print('Reg coefs:')
LinearRegression(fit_intercept=False).fit(data2[:,:3],data2[:,3]).coef_

[[  0   1   0 850   4]
 [  0   1   0 650   3]
 [  0   0   1 700   1]
 [  0   0   1 650   3]
 [  1   0   0 700   3]
 [  1   0   0 600   2]] 

['neighborhood=Fremont', 'neighborhood=Queen Anne', 'neighborhood=Wallingford', 'price', 'rooms']
Reg coefs:


array([650., 750., 675.])

# Imputation/Missing Values  

# Modeling missing data, using the A6 dataset 

In [33]:
import pandas as pd 
import seaborn as sns
import numpy as np
from statsmodels.formula.api import ols as sm_ols

In [34]:
url = 'https://github.com/LeDataSciFi/lectures-spr2020/blob/master/assignment_data/Fannie_Mae_Plus_Data.gzip?raw=true'
fannie_mae = pd.read_csv(url,compression='gzip')  
print(fannie_mae.info())
pd.options.display.float_format = '{:,.1f}'.format
fannie_mae.drop('Loan_Identifier',axis=1).describe().round(1).transpose()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135038 entries, 0 to 135037
Data columns (total 36 columns):
Loan_Identifier                            135038 non-null float64
Origination_Channel                        135038 non-null object
Seller_Name                                135038 non-null object
Original_Interest_Rate                     135038 non-null float64
Original_UPB                               135038 non-null float64
Original_Loan_Term                         135038 non-null float64
Original_LTV_(OLTV)                        135038 non-null float64
Original_Combined_LTV_(CLTV)               134007 non-null float64
Number_of_Borrowers                        135007 non-null float64
Original_Debt_to_Income_Ratio              132396 non-null float64
Borrower_Credit_Score_at_Origination       134481 non-null float64
Loan_purpose                               135038 non-null object
Property_type                              135038 non-null object
Number_of_units       

,count,mean,std,min,25%,50%,75%,max
Original_Interest_Rate,"135,038.0",5.2,1.3,2.2,4.2,5.2,6.1,11.0
Original_UPB,"135,038.0","188,931.1","108,742.4","8,000.0","108,000.0","164,000.0","247,000.0","1,170,000.0"
Original_Loan_Term,"135,038.0",307.1,82.3,60.0,240.0,360.0,360.0,360.0
Original_LTV_(OLTV),"135,038.0",70.1,17.5,4.0,60.0,75.0,80.0,97.0
Original_Combined_LTV_(CLTV),"134,007.0",70.9,17.6,4.0,61.0,75.0,80.0,142.0
Number_of_Borrowers,"135,007.0",1.6,0.5,1.0,1.0,2.0,2.0,8.0
Original_Debt_to_Income_Ratio,"132,396.0",33.3,11.5,1.0,25.0,33.0,42.0,64.0
Borrower_Credit_Score_at_Origination,"134,481.0",742.4,53.4,361.0,707.0,755.0,786.0,850.0
Number_of_units,"135,038.0",1.0,0.2,1.0,1.0,1.0,1.0,4.0
Zip_code_short,"135,038.0",546.0,306.0,0.0,295.0,549.0,844.0,999.0


In [35]:
# "train" an estimator using credit score 
fannie_mae['l_coscore'] = np.log(fannie_mae['Co-borrower_credit_score_at_origination'])
sm_ols('Original_Interest_Rate ~ l_coscore', data=fannie_mae).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.122
Model:                                OLS   Adj. R-squared:                  0.122
Method:                     Least Squares   F-statistic:                     9348.
Date:                    Tue, 07 Apr 2020   Prob (F-statistic):               0.00
Time:                            14:21:06   Log-Likelihood:            -1.0870e+05
No. Observations:                   67366   AIC:                         2.174e+05
Df Residuals:                       67364   BIC:                         2.174e+05
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     47.8399      0.442    108.267      0.000      46.974      48.706
l_coscore     -6.4539      0.067    -96.685      0.000      -6.585      -6.323
==============================================================================
Omnibus:                     1629.968   Durbin-Watson:                   0.393
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1667.489
Skew:                           0.365   Prob(JB):                         0.00
Kurtosis:                       2.753   Cond. No.                         639.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
# treat all missing values as a seperate category 

In [37]:
# if i am willing to assume that all missing credit scores are avg: 

fannie_mae ['missing_coscore'] =fannie_mae ['l_coscore'].mean()


In [38]:
mean_coscore = fannie_mae['l_coscore'].mean()
fannie_mae['l_coscore'] = fannie_mae['l_coscore'].fillna(mean_coscore)

In [39]:
fannie_mae['l_coscore'].describe()
sm_ols('Original_Interest_Rate ~ l_coscore', data=fannie_mae).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.061
Model:                                OLS   Adj. R-squared:                  0.061
Method:                     Least Squares   F-statistic:                     8834.
Date:                    Tue, 07 Apr 2020   Prob (F-statistic):               0.00
Time:                            14:21:19   Log-Likelihood:            -2.2171e+05
No. Observations:                  135038   AIC:                         4.434e+05
Df Residuals:                      135036   BIC:                         4.434e+05
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     47.9578      0.455    105.510      0.000      47.067      48.849
l_coscore     -6.4539      0.069    -93.988      0.000      -6.588      -6.319
==============================================================================
Omnibus:                     3145.470   Durbin-Watson:                   0.315
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2712.363
Skew:                           0.286   Prob(JB):                         0.00
Kurtosis:                       2.607   Cond. No.                         905.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# impute class- similar to clear vectorizer- fit the transformer on to the data 

# Standardization 

In [ ]:
# important to have documentation and alot of their models 
# if you have variable has alot of variation shows more impact than actually does 
# after you standardize you apply preprocessing module 

In [40]:
# a very simple example
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
X_scaled = preprocessing.scale(X_train)

print(' X_scaled\n',         '-'*40,'\n',X_scaled,'\n')
print(' Mean of each var:\n','-'*40,'\n',X_scaled.mean(axis=0),'\n')
print(' STD of each var:\n', '-'*40,'\n',X_scaled.std(axis=0),'\n')

#  standardization <1 and turn all the varibale into main 0 

 X_scaled
 ---------------------------------------- 
 [[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]] 

 Mean of each var:
 ---------------------------------------- 
 [0. 0. 0.] 

 STD of each var:
 ---------------------------------------- 
 [1. 1. 1.] 

